In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import pandas as pd

In [ ]:
dataset = pd.read_csv('traindata.csv')
dropped = ['processed', 'offensiveness_score']
rename = {'txt': 'comment', 'isOffensive': 'label'}
dataset = dataset.drop(columns=dropped)
dataset = dataset.rename(columns=rename)
dataset.head()